In [ ]:
# Test importing of video in opencv
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
%matplotlib widget

# Stored video file location
fdir = '/home/csarantos/cattown/'
fname = '2024-03-22 first trial.mov'

cv.destroyAllWindows()

In [ ]:
# Some helper functions to analyze the moving object contour

def get_centroid(contour):
    ''' Get centroid of an openCV 2d contour '''
    row = int( np.round( contour[:,0,0].mean() ) )
    col = int( np.round( contour[:,0,1].mean() ) )
    return (row, col)

def get_2ndmoments(contour, centroid):
    ''' Get moments of inertia along x and y, return vector - NOT CURRENTLY USED '''
    cr = np.reshape(contour, (contour.shape[0], 2))

    dcr = cr - centroid
    Iy = np.sum( (dcr[:,0]**2) / dcr.shape[0] )
    Ix = np.sum( (dcr[:,1]**2) / dcr.shape[0] )

    return (round(Iy), round(Ix)) # row, col    

def get_max_variance_vectors(contour, centroid):
    cr = np.reshape(contour, (contour.shape[0], 2))

    dcr = cr - centroid
    # Not sure if this is needed, scale to unit variance
    # Actually it hurts us here, we want to keep info on which direction has more spatial extent
    # dcr[:,0] = dcr[:,0] / (dcr[:,0].std())
    # dcr[:,1] = dcr[:,1] / (dcr[:,1].std())

    # NOTE: A PCA approach would be to define a dependent variable, distance from centroid,
    # and treat X-cent_x and Y-cent_x as independent variables, and do mini-2d-PCA that way
    # Here we're just seeing how Dx and Dy covary with each other,
    # and trying to find the tilted axis that explains the most variance in both x and y,
    # in other words, the line thru the longest dimension of the arbitrarily rotated shape, which is what we want

    # The eigenvector of the covariance matrix with the larger eigenvalue is the direction of max variance
    covariance_mat = np.cov(dcr.T)
    eigenvalues, eigenvectors = np.linalg.eig(covariance_mat)

    # return the eigenvectors scaled by the eigenvalues (amount of variance along each axis)
    return [eigenvectors[:,i] * ev for i, ev in enumerate(eigenvalues)]

In [ ]:
# cr = np.reshape(contour, (contour.shape[0], 2))

# dcr = cr - centroid
# # Not sure if this is needed, scale to unit variance
# dcr[:,0] = dcr[:,0] / (dcr[:,0].std())
# dcr[:,1] = dcr[:,1] / (dcr[:,1].std())

# covariance_mat = np.cov(dcr.T)
# #
# # Calculate Eigenvalues and Eigenmatrix
# #
# eigenvalues, eigenvectors = np.linalg.eig(covariance_mat)
# tuple(eigenvectors[ eigenvalues.argmax() ])

In [ ]:
WRITE_OUTPUT = True
READ_FILE = True
read_livecam_id = 0
SHOW_ORIENTATION = True

# Magic numbers:
# minimum object contour area to be tracked:
# min_contour_area = 500  
min_contour_area = 2000

backsub_thresh = 180 # Threshold for movement in background subtracted image

# Process only subset of video:
frame_start = 300
frames_to_proc = 450

# Amounts to scale direction vector..
# maxvar_scaling = 50 # old method
maxvar_scaling = 0.02 # Since we don't divide x,y by their stdev, things get large

# Attempt manual mtion tracking method - background subtraction and contours:
back_sub = cv.createBackgroundSubtractorMOG2()

frame_end = frame_start + frames_to_proc
frame_num = 0

if READ_FILE:
    cap = cv.VideoCapture( os.path.join(fdir, fname))
    # Advance the capture to skip the first frames we don't want:
    cap.set(cv.CAP_PROP_POS_FRAMES, frame_start)
else:
    frames_to_proc = np.inf
    cap = cv.VideoCapture(read_livecam_id)

# Configure output stream:
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
# out_fn = fname.replace('.mov', '_Annotated.mp4v')
out_fn = fname.replace('.mov', '_test2Annotated.avi')

if WRITE_OUTPUT:
    out = cv.VideoWriter(os.path.join(fdir, out_fn),
                          cv.VideoWriter_fourcc('M','P','4','V'), 24, (frame_width,frame_height))
    
    if not out.isOpened():
        raise ValueError('VideoWriter failed to initialize')

while cap.isOpened():
    frame_num += 1
    
    if frame_num > frame_end:
        break
    
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # Apply background subtraction
    fg_mask = back_sub.apply(frame)

    # apply global threshold to remove shadows
    retval, mask_thresh = cv.threshold( fg_mask, backsub_thresh, 255, cv.THRESH_BINARY)

    # Binary erosion to remove small motion pixel groups
    # set the kernal
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
    # Apply erosion
    mask_eroded = cv.morphologyEx(mask_thresh, cv.MORPH_OPEN, kernel)

    # Find contours
    contours, hierarchy = cv.findContours(mask_eroded, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)    
 
    # Filter contours to allow only larger objects:
    large_contours = [cnt for cnt in contours if cv.contourArea(cnt) > min_contour_area]
    centroids = [get_centroid(c) for c in large_contours]
    
    # Annotated frame:
    frame_ct = cv.drawContours(frame, large_contours, -1, (0, 255, 0), 2)
    
    # Draw centroids with extra markers if we're not using orientation markers:
    if not SHOW_ORIENTATION:
        for c in centroids:
            frame_ct = cv.drawMarker(frame_ct, c, (0, 0, 255), markerType=cv.MARKER_TILTED_CROSS, 
                                     markerSize=15, thickness=2, line_type=cv.LINE_AA)
    else:
        # Draw eigenvectors of the covariance matrix scaled by eigenvalues
        # Gets across "direction" axis of variance, and if they're equally sized it's symmetric
        for c, cent in zip(large_contours, centroids):
            eigenvecs = get_max_variance_vectors(c, cent)
            for ev in eigenvecs:
                vary = round(maxvar_scaling*ev[0])
                varx = round(maxvar_scaling*ev[1])
                # Sign of max variance vector is arbitrary, so draw a line in both + and - directions from centroid
                frame_ct = cv.line( frame_ct, (cent[0]-vary, cent[1]-varx), (cent[0]+vary, cent[1]+varx), (0, 0, 255), 2 )
    
    # Display the resulting frame
    cv.imshow('Frame_final', frame_ct)

    if WRITE_OUTPUT:
        # Write it to the video output file:
        out.write(frame_ct)
    
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
if WRITE_OUTPUT:
    out.release()
    
cv.destroyAllWindows()

In [ ]:
eigenvecs

In [ ]:
# scratch
# c = contours[6]
# cr = np.reshape(c, (c.shape[0], 2))
# cr.shape

# disp_scaling = 10

# centroid = get_centroid(c)
# dcr = cr - centroid
# norms = np.linalg.norm(dcr, axis=1).reshape((cr.shape[0],1))
# dcrnorm = dcr / norms
# diry = dcrnorm[:,0].mean()
# dirx = dcrnorm[:,1].mean()
# disp_dirvec = ( round( disp_scaling*diry ), round( disp_scaling*dirx ) )

In [ ]:
# Play back video with no annotations:

while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
        
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cv.imshow('frame', gray)
    if cv.waitKey(1) == ord('q'):
        break
            
cap.release()
cv.destroyAllWindows()